In [93]:
class Cost_mat():
    
    def __init__(self, seq, arr):
        import numpy as np
        arr=np.array(arr)
        if sum([seq.count(ch) for ch in seq])!=len(seq):
            raise ValueError('Sequnce contains repeating characters')
        elif len(arr.shape)!=2 or arr.shape[0]!=arr.shape[1]:
            raise ValueError('Array is not a two-dimensional square array')
        elif len(seq)!=arr.shape[0]:
            raise ValueError('Array dimension does not match sequence length')
        elif '-' not in seq:
            raise ValueError('Sequnce must contain "-" sign')
        else:
            self.characters=seq
            self.matrix=np.array(arr)
        
    def acces(self, ch_1, ch_2):
        i=self.characters.index(ch_1)
        j=self.characters.index(ch_2)
        return self.matrix[i][j]
    

In [11]:
def Needleman_Wunsch(seq_1, seq_2, indel=-1, missmatch=-1, match=1, cost_mat=None):
    import numpy as np
    import itertools

    def cost_func(el_1, el_2, indel=indel, missmatch=missmatch, match=match, cost_mat=cost_mat):
        if cost_mat:
            return cost_mat.acces(el_1, el_2)
        elif el_1=='-' or el_2=='-':
            return indel
        else:
            return match if el_1==el_2 else missmatch
    
    Dist_mat=np.empty([len(seq_1)+1, len(seq_2)+1])
    Path_mat=np.empty([len(seq_1)+1, len(seq_2)+1])
    
    Dist_mat[:,0]=[x*indel for x in range(0, len(seq_1)+1)]
    Dist_mat[0,:]=[x*indel for x in range(0, len(seq_2)+1)]
    
    Path_mat[:,0]=[0]*(len(seq_1)+1)
    Path_mat[0,:]=[1]*(len(seq_2)+1)

    for i, j in itertools.product(range(1, len(seq_1)+1), range(1, len(seq_2)+1)):
    
        Dist_mat[i, j]=max(Dist_mat[i-1,j]+cost_func(seq_1[i-1], '-'), Dist_mat[i, j-1]+cost_func('-', seq_2[j-1]), 
                           Dist_mat[i-1,j-1]+cost_func(seq_1[i-1], seq_2[j-1]))
        
        Path_mat[i, j]=np.argmax([Dist_mat[i-1,j]+cost_func(seq_1[i-1], '-'), Dist_mat[i, j-1]+cost_func('-', seq_2[j-1]), 
                           Dist_mat[i-1,j-1]+cost_func(seq_1[i-1], seq_2[j-1])])

    Alignment=[[],[]]
    
    i,j=len(seq_1),len(seq_2)
    
    while (i,j)!=(0,0):
        
        if Path_mat[i, j]==0: #Deletion
            Alignment[0].append(seq_1[i-1])
            Alignment[1].append('-')
            i-=1
        elif Path_mat[i, j]==1: #Insertion
            Alignment[0].append('-')
            Alignment[1].append(seq_2[j-1])
            j-=1
        else: #Match/Missmatch
            Alignment[0].append(seq_1[i-1])
            Alignment[1].append(seq_2[j-1])
            i-=1
            j-=1
            
        
    Alignment[0]=Alignment[0][::-1]
    Alignment[1]=Alignment[1][::-1]
    return Dist_mat, Alignment

In [94]:
def Smith_Waterman(seq_1, seq_2, indel=-1, missmatch=-1, match=1, cost_mat=None):
    import numpy as np
    import itertools

    def cost_func(el_1, el_2, indel=indel, missmatch=missmatch, match=match, cost_mat=cost_mat):
        if cost_mat:
            return cost_mat.acces(el_1, el_2)
        elif el_1=='-' or el_2=='-':
            return indel
        else:
            return match if el_1==el_2 else missmatch
    
    Dist_mat=np.empty([len(seq_1)+1, len(seq_2)+1])
    Path_mat=np.empty([len(seq_1)+1, len(seq_2)+1])
    
    Dist_mat[:,0]=[0]*(len(seq_1)+1)
    Dist_mat[0,:]=[0]*(len(seq_2)+1)
    
    Path_mat[:,0]=[0]*(len(seq_1)+1)
    Path_mat[0,:]=[1]*(len(seq_2)+1)

    for i, j in itertools.product(range(1, len(seq_1)+1), range(1, len(seq_2)+1)):
    
        Dist_mat[i, j]=max(Dist_mat[i-1,j]+cost_func(seq_1[i-1], '-'), Dist_mat[i, j-1]+cost_func('-', seq_2[j-1]), 
                           Dist_mat[i-1,j-1]+cost_func(seq_1[i-1], seq_2[j-1]), 0)
        
        Path_mat[i, j]=np.argmax([Dist_mat[i-1,j]+cost_func(seq_1[i-1], '-'), Dist_mat[i, j-1]+cost_func('-', seq_2[j-1]), 
                           Dist_mat[i-1,j-1]+cost_func(seq_1[i-1], seq_2[j-1])])
      
    Local_Alignment=[[],[]]
    
    flag_1=True
    flag_2=True
    while (i,j)!=(0,0):
        
        if Dist_mat[i,j]==np.max(Dist_mat) and flag_1:
            Local_Alignment[0].append('*')
            Local_Alignment[1].append('*')
            flag_1=False
        if Dist_mat[i,j]==0 and flag_2:
            Local_Alignment[0].append('*')
            Local_Alignment[1].append('*')
            flag_2=False
        
        if Path_mat[i, j]==0: #Deletion
            Local_Alignment[0].append(seq_1[i-1])
            Local_Alignment[1].append('-')
            i-=1
        elif Path_mat[i, j]==1: #Insertion
            Local_Alignment[0].append('-')
            Local_Alignment[1].append(seq_2[j-1])
            j-=1
        else: #Match/Missmatch
            Local_Alignment[0].append(seq_1[i-1])
            Local_Alignment[1].append(seq_2[j-1])
            i-=1
            j-=1
                
    Local_Alignment=[x[::-1] for x in Local_Alignment]
    
    return Dist_mat, Local_Alignment

## Fixed constant penalties
### Same length sequences, their alignment contains both gaps and missmatches

In [12]:
D, A=Needleman_Wunsch('EDTNG', 'DSTNC', -1, -1, 1)

In [13]:
print(A[0])
print(A[1])

['E', 'D', '-', 'T', 'N', 'G']
['-', 'D', 'S', 'T', 'N', 'C']


### Different weights

In [14]:
D_m, A_m=Needleman_Wunsch('EDTNG', 'DSTNC', -0.499, -1, 1)

In [15]:
print(A_m[0])
print(A_m[1])

['E', 'D', '-', 'T', 'N', '-', 'G']
['-', 'D', 'S', 'T', 'N', 'C', '-']


As expected, the algoritm preffers a pair of insertion and deletion with overall weight of -0.998 to a single missmatch with a weight of -1.

## Alignment with substitution weight matrix

In [4]:
C=Cost_mat('-ABC', ([[1,-1,-1,-1], [-1, 1, -0.8, -1.5], [-1, -0.8, 1, -2], [-1, -1.5, -2, 1]]))

In [24]:
D_c, A_c=Needleman_Wunsch('ABCAAA','ABCBACB', cost_mat=C)

In [25]:
print(A_c[0])
print(A_c[1])

['A', 'B', 'C', 'A', 'A', '-', 'A']
['A', 'B', 'C', 'B', 'A', 'C', 'B']


Let's change a single value in the weight matrix so the optimal alignment would change.

In [26]:
C_m=Cost_mat('-ABC', ([[1,-1,-1,-1], [-1, 1, -10, -1.5], [-1, -0.8, 1, -2], [-1, -1.5, -2, 1]]))

In [29]:
D_c, A_c=Needleman_Wunsch('ABCAAA','ABCBACB', cost_mat=C_m)

In [30]:
print(A_c[0])
print(A_c[1])

['A', 'B', 'C', '-', 'A', 'A', '-', 'A']
['A', 'B', 'C', 'B', 'A', 'C', 'B', '-']


## Local alignment

In [95]:
LD, LA=Smith_Waterman('PWAPSQAABCABCCDACLWO', 'PWAKKSABCCDCSKS')
ND, NA=Needleman_Wunsch('PWAPSQAABCABCCDACLWO', 'PWAKKSABCCDCSKS')

In [96]:
print(NA[0])
print(NA[1])

['P', 'W', 'A', 'P', '-', 'S', 'Q', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'D', 'A', 'C', 'L', 'W', 'O']
['P', 'W', 'A', 'K', 'K', 'S', '-', 'A', '-', 'B', 'C', '-', '-', 'C', '-', 'D', '-', 'C', 'S', 'K', 'S']


Star signs * mark local alignment.

In [97]:
print(LA[0])
print(LA[1])

['P', 'W', 'A', 'P', '-', 'S', 'Q', 'A', 'A', 'B', 'C', '*', 'A', 'B', 'C', 'C', 'D', 'A', 'C', '*', 'L', 'W', 'O']
['P', 'W', 'A', 'K', 'K', 'S', '-', '-', '-', '-', '-', '*', 'A', 'B', 'C', 'C', 'D', '-', 'C', '*', 'S', 'K', 'S']
